In [ ]:
!pip install -q trino

In [ ]:
import requests
from IPython.display import JSON

In [ ]:
CATALOG_URL = "http://lakekeeper:8181/catalog"
MANAGEMENT_URL = "http://lakekeeper:8181/management"
TRINO_URI = "https://trino-proxy:443"
KEYCLOAK_TOKEN_URL = "http://keycloak:8080/realms/iceberg/protocol/openid-connect/token"
WAREHOUSE = "test-warehouse"

## 1. Sign in

In [ ]:
CLIENT_ID = "trino"
CLIENT_SECRET = "AK48QgaKsqdEpP9PomRJw7l2T7qWGHdZ"

response = requests.post(
    url=KEYCLOAK_TOKEN_URL,
    data={
        "grant_type": "client_credentials",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "scope": "lakekeeper"
    },
    headers={"Content-type": "application/x-www-form-urlencoded"},
)

access_token = response.json()['access_token']

In [ ]:
# Force lakekeeper create a user for "service account trino"

response = requests.post(
    url=f"{MANAGEMENT_URL}/v1/user",
    headers={"Authorization": f"Bearer {access_token}"},
    json={"update-if-exists": True}
)
response.text

In [ ]:
# Check our lakekeeper user
response = requests.get(
    url=f"{MANAGEMENT_URL}/v1/whoami",
    headers={"Authorization": f"Bearer {access_token}"},
)
response.raise_for_status()
JSON(response.json())

In [ ]:
# Now wait for an admin to assign permissions to Trino

In [ ]:
# Go back to Management notebook or connect with cdo on UI...

# Connect to Trino

To create catalog, query must be done by a human admin: `cdo`. Deconnect from existsing lakekeeper tab or open trino token rediect url into private browser window and connect as cdo: `cdo`/`iceberg`

In [ ]:
from trino.dbapi import connect
from trino.auth import OAuth2Authentication
from redirect_handler import REDIRECT_HANDLER
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

conn = connect(
    host=TRINO_URI,
    auth=OAuth2Authentication(REDIRECT_HANDLER),
    http_scheme="https",
    verify=False,
    catalog="lakekeeper"
)

In [ ]:
cur = conn.cursor()
print(cur.execute("SELECT current_user").fetchall())  # should return cdo keycloak user id

# Creating the Trino Catalog
Make sure to follow the instructions in the `01-Bootstrap.ipynb` and `02-Create-Warehouse.ipynb` notebooks first!

In [ ]:
create_catalog_sql = f"""
    CREATE CATALOG lakekeeper USING iceberg
    WITH (
        "iceberg.catalog.type" = 'rest',
        "iceberg.rest-catalog.uri" = '{CATALOG_URL}',
        "iceberg.rest-catalog.warehouse" = '{WAREHOUSE}',
        "iceberg.rest-catalog.security" = 'OAUTH2',
        "iceberg.rest-catalog.oauth2.credential" = '{CLIENT_ID}:{CLIENT_SECRET}',
        "iceberg.rest-catalog.vended-credentials-enabled" = 'true',
        "iceberg.rest-catalog.oauth2.scope" = 'lakekeeper',
        "iceberg.rest-catalog.oauth2.server-uri" = '{KEYCLOAK_TOKEN_URL}',
        "s3.region"= 'local',
        "s3.path-style-access" = 'true',
        "s3.endpoint" = 'http://minio:9000',
        "fs.native-s3.enabled" = 'true'
    )
"""

In [ ]:
print(create_catalog_sql)

In [ ]:
cur = conn.cursor()
cur.execute(create_catalog_sql)

In [ ]:
# Make sure lakekeeper is created
print(cur.execute("SHOW catalogs").fetchall())